In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool, create_tool_calling_agent, AgentExecutor
import gradio as gr
from answer import answer_question
from langchain.chat_models import ChatOpenAI
import google.generativeai as genai
from langchain.llms.base import LLM
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

d:\LangChain Projects\langchain_new\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\LangChain Projects\Multi_Agents_Flight_Intelligence\3_Help_Flight_Agent\answer.py:38: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  openrouter_chat = ChatOpenAI(model=os.getenv("DEEPSEEK_MODEL"),


In [2]:
load_dotenv(override=True)

gemini_api_key = os.getenv("GEMINI_API_KEY_M")
gemini_model = os.getenv("GEMINI_MODEL")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
gpt_model = os.getenv("GPT_MODEL")


In [3]:
# gemini_chat = ChatGoogleGenerativeAI(
    # model=gemini_model,  
    # temperature=0,
    # max_output_tokens=2048,  
    # google_api_key=gemini_api_key,
    # convert_system_message_to_human=True  
# )

In [4]:
# class GoogleGenAIChat(LLM, BaseModel):
    # model: str = Field(default="gemini-1.5-flash")
    # temperature: float = Field(default=0)
    # max_output_tokens: int = Field(default=2048)
    # convert_system_message_to_human: bool = Field(default=True)
# 
    # class Config:
        # arbitrary_types_allowed = True  # allows storing non-Pydantic objects if needed
# 
    # @property
    # def _llm_type(self):
        # return "google-genai"
# 
    # def _call(self, prompt: str, stop=None) -> str:
        # if self.convert_system_message_to_human:
            # prompt = prompt.replace("System:", "").strip()
# 
        # response = genai.generate_text(
            # model=self.model,
            # prompt=prompt,
            # temperature=self.temperature,
            # max_output_tokens=self.max_output_tokens
        # )
        # return response.text

In [5]:
# gemini_chat = GoogleGenAIChat(
    # model="gemini-2.0-flash",
    # temperature=0,
    # max_output_tokens=2048,
    # convert_system_message_to_human=True
# )

In [6]:
openrouter_chat = ChatOpenAI(model=os.getenv("DEEPSEEK_MODEL"),
                             api_key=openrouter_api_key,
                             max_tokens=1000,
                             temperature=0,
                             base_url="https://openrouter.ai/api/v1")

In [7]:
chat_memory = ConversationBufferMemory(
                                        memory_key="chat_history",
                                        return_messages=True)

C:\Users\Ali Sena\AppData\Local\Temp\ipykernel_7480\230103001.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationBufferMemory(


In [8]:
retrieval_tool = Tool(
    name="retrieval_tool",
    func=answer_question,
    description="""
    This tool answers any question related to the flight company SkyVista Airlines using a Retrieval-Augmented Generation (RAG) approach. 
    It retrieves relevant documents from the knowledge base, combines them with conversation history if provided, 
    and generates a context-aware answer. The tool returns both the generated answer and the source chunks used for reference.
    """
)


In [9]:
system_prompt = SystemMessagePromptTemplate.from_template("""
You are a helpful and accurate assistant for SkyVista, a flight company. 
Your task is to answer any questions about SkyVista using the provided retrieval tool called `retrieval_tool`. 

Rules for answering questions:

1. Always use information retrieved from the `retrieval_tool` to answer questions.
2. Do not invent facts about SkyVista. If the answer is not in the retrieved documents, politely say that the information is not available.
3. Keep answers clear, concise, and relevant to the question.
4. Consider conversation history to provide context-aware responses when available.
5. Provide useful details like flight schedules, policies, services, or other official company info only if confirmed by the documents.

Remember, your primary source of truth is the `retrieval_tool` that fetches chunks from the SkyVista knowledge base.
""")


In [10]:
tools = [retrieval_tool]

In [11]:
chat_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{input}")
])


In [12]:
agent_executor = initialize_agent(
    tools=tools,
    llm=openrouter_chat,
    agent="chat-conversational-react-description",
    memory=chat_memory,
    verbose=True,
    agent_kwargs={"prompt": chat_prompt}   # ← THIS is key
)


C:\Users\Ali Sena\AppData\Local\Temp\ipykernel_7480\2789510515.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [13]:
def ask_retrieval_agent(query, history):
    response = agent_executor.run(input=query)
    return response

In [ ]:
gr.ChatInterface(fn=ask_retrieval_agent, 
                 title="Retrieval Assistant").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


C:\Users\Ali Sena\AppData\Local\Temp\ipykernel_7480\2055105201.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run(input=query)




> Entering new AgentExecutor chain...


```json
{
    "action": "retrieval_tool",
    "action_input": "tell me about the company"
}
```
Observation: ('\n\nIt seems there might be a slight confusion! I represent **Insurellm**, but the context you provided relates to **SkyVista Airlines**. Since I don’t have specific details about Insurellm in the context, I’ll share information about SkyVista Airlines based on what’s provided. If you meant to ask about Insurellm instead, let me know, and I’ll clarify further!  \n\n---\n\n### **About SkyVista Airlines**  \nSkyVista Airlines is a mid-sized, privately held airline headquartered in **Dubai, UAE**, with regional offices in Karachi, Istanbul, and Kuala Lumpur. Founded in **2016**, it focuses on connecting South Asia, the Middle East, Southeast Asia, and select European destinations.  \n\n#### **Key Highlights**:  \n- **Leadership**:  \n  - **CEO**: Ali Sina Ghulami (oversees strategy and growth).  \n  - **COO**: Sarah K. Mirza (manages day-